In [37]:
import pandas as pd
import numpy as np

In [49]:
columns_names = ['session','client','?','choice'] + ["option_0"+str(i) for i in range(1,4)] + ["option_1"+str(i) for i in range(1,4)]

data = pd.read_csv("data_for_matlab.csv",header=None,index_col=None, names=columns_names )
data

,session,client,?,choice,option_01,option_02,option_03,option_11,option_12,option_13
0,5,0,1,0,105,105,80,32,144,80
1,5,0,2,1,21,95,80,6,130,80
2,5,0,3,1,125,85,80,52,124,80
3,5,0,4,0,41,75,80,26,110,80
4,5,0,5,1,145,65,80,72,104,80
...,...,...,...,...,...,...,...,...,...,...
1411,6,23,34,1,21,95,150,6,130,150
1412,6,23,35,0,125,85,150,52,124,150
1413,6,23,36,0,41,75,150,26,110,150
1414,6,23,37,1,145,65,150,72,104,150


In [59]:
#u=@(a,d,o) a*mean(d(:,5+o*3:7+o*3),2)-(1-a)*std(d(:,5+o*3:7+o*3),1,2);
def u(a,d,o):
    columns = ["option_" +str(int(o)) +str(i) for i in range(1,4)]
    print(d[columns].mean(axis=1))
    return a*d[columns].mean(axis=1)+(1-a)*d[columns].std(axis=1)

def data_pred(a,d):
    return 1*(u(a,data,0) > u(a,data,1)) #+ .5*(u(a,data,0) == u(a,data,1))

def data_sc(a,d):
    predictions = data_pred(a,d)
    return 1*(data['choice']==predictions) #+ .5*(.5==predictions)

print(data_sc(.2,data).mean())

0        96.666667
1        65.333333
2        96.666667
3        65.333333
4        96.666667
           ...    
1411     88.666667
1412    120.000000
1413     88.666667
1414    120.000000
1415     88.666667
Length: 1416, dtype: float64
0        85.333333
1        72.000000
2        85.333333
3        72.000000
4        85.333333
           ...    
1411     95.333333
1412    108.666667
1413     95.333333
1414    108.666667
1415     95.333333
Length: 1416, dtype: float64
0.4837570621468927


In [ ]:
# results matrices
# ---------------------------
res(1:24+36,1:num_as)=NaN;
res_cl(1:24+36,1:2)=NaN;
res_row=0;
for client=0:35
    res_row=res_row+1;
    score=sum(data_sc(1+client*14:14+client*14,:),1)/14;
    session=5;
    res(res_row,:)=score;
    res_cl(res_row,1:2)=[session,client];
    clear score session
end
for client=0:23
    res_row=res_row+1;
    score=sum(data_sc(504+1+client*38:504+38+client*38,:),1)/38;
    session=6;
    res(res_row,:)=score;
    res_cl(res_row,1:2)=[session,client];
    clear score session
end
clear client res_row

#maximum scores
# ---------------------------
res_max(1:24+36,1:5)=NaN;
for res_row=1:60
    [max_val,max_i]=max(res(res_row,:));
    res_max(res_row,1)=max_val;
    res_max(res_row,2)=size(find(res(res_row,:)==max_val),2);
    res_max(res_row,3)=A(min(find(res(res_row,:)==max_val)),1);
    res_max(res_row,4)=mean(A(find(res(res_row,:)==max_val)));
    res_max(res_row,5)=A(max(find(res(res_row,:)==max_val)),1);
    clear max_val max_i
end
clear res_row

In [ ]:
if max_k>=1
    kvars1=combntns(A,1);
    kvars1(:,2:4)=NaN;
    kvars1a(1:size(kvars1,1),1:2)=NaN;
    for i=1:size(kvars1,1)
        k1=kvars1(i,1);
        k1i=find(A==k1);
        kvars1(i,2)=60;
        kvars1(i,3)=mean(res(:,k1i));
        kvars1(i,4)=mean(res(:,k1i));
        kvars1a(i,1)=60;
        kvars1a(i,2)=mean(res(:,k1i));;
        clear k1 k1i
    end
    clear i
    [kmaxv(1,1) kmaxi(1,1)]=max(kvars1(:,3));
    disp(['optimal scores k=1 done']);diary off;diary on;
end